In [20]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
import datetime
import matplotlib.pyplot as plt
import collections as cc
import seaborn as sns
from collections import Counter
import re

In [43]:
data_dir='processed_data_with_view.csv'
data=pd.read_csv(data_dir,encoding='utf-8',index_col=0)

In [44]:
data

,id,status,create_date,name,keyword,price,flag_used,flag_taekpo,flag_exchg,qty,...,pid,first_view_date,purchase_date,create_to_view_duration,create_to_talk_duration,create_to_purchase_duration,view_to_talk_duration,view_to_purchase_duration,talk_to_purchase_duration,total_view
0,0eb2b66585eefd1d57a21397ca0681e4,3,2024-02-05 12:02:00,[택포] [280] 나이키 리액티 인피니티 런 플라이니트 런닝화,"나이키리액트,나이키런,나이키런닝화,나이키러닝화,플라이니트",35000,20,1,0,0,...,0eb2b66585eefd1d57a21397ca0681e4,2024-02-05,2024-02-13 09:07:00,0.0,8.0,11345.0,8.0,8.0,0.0,107
1,bd17645bfa0ff30741d56cf9efbfc609,3,2024-01-26 12:40:00,아디다스 리프시몬스 275 박스풀 미개봉,라프시몬스,130000,2,1,0,1,...,bd17645bfa0ff30741d56cf9efbfc609,2024-01-26,2024-01-29 23:47:00,0.0,2.0,4987.0,2.0,3.0,1.0,13
2,96562ef11a40da0b446e34b056ca0132,2,2023-12-24 08:55:00,(새상품) 아디다스 예쁜 운동화 (정가는 249천원),"아디다스250,운동화255,260사이즈,아디다스270",69000,2,0,0,6,...,96562ef11a40da0b446e34b056ca0132,2023-12-24,2024-01-06 11:11:00,0.0,2.0,18856.0,2.0,13.0,11.0,1348
3,59beb5293ca3dccb43a70f01324775e1,3,2024-04-28 17:55:00,(280) 뉴발란스 860v2 에임레온도르 그린,"뉴발란스860v2,에메레온도르,에임레온도르",500000,20,1,0,0,...,59beb5293ca3dccb43a70f01324775e1,2024-04-28,2024-04-29 08:51:00,0.0,1.0,896.0,1.0,1.0,0.0,12
4,f43ce1e517aff9ff286967aa98d1d22a,3,2024-04-26 09:42:00,나이키 스투시 에어맥스 2013 싸이킥 핑크 (240),NaN,230000,21,0,0,0,...,f43ce1e517aff9ff286967aa98d1d22a,2024-04-26,2024-04-26 09:48:00,0.0,0.0,6.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23910,00c0b941c600fd57d348394f0ff14027,3,2024-01-20 18:21:00,리복 퓨리 250판매,"리복퓨리,리복,250",20000,22,0,0,0,...,00c0b941c600fd57d348394f0ff14027,2024-01-20,2024-01-20 18:48:00,0.0,0.0,27.0,0.0,0.0,0.0,0
23923,0f8ebfca121d7907b429b7a12fa147e4,3,2024-04-24 18:11:00,220)새상품/ 우먼 나이키 프리런 2018 /정품/원가이하세일,"나이키프리런,프리런2018,나이키런닝화,런닝화220,런닝화215",55000,2,1,0,0,...,0f8ebfca121d7907b429b7a12fa147e4,2024-04-25,2024-05-12 23:32:00,1.0,18.0,26241.0,17.0,17.0,0.0,9
23924,a4aaa33329298f6689062f010ed43012,3,2024-04-08 12:16:00,조던 농구화 285,NaN,50000,2,1,0,0,...,a4aaa33329298f6689062f010ed43012,2024-04-08,2024-04-08 18:11:00,0.0,0.0,355.0,0.0,0.0,0.0,0
23925,183cb70958de0f324877ea890eedc3f9,3,2024-04-06 11:02:00,뉴발란스 x90 x-90 270mm,"newbalanc,뉴발란스,x90,x-90",55000,21,0,0,1,...,183cb70958de0f324877ea890eedc3f9,2024-04-06,2024-04-11 05:44:00,0.0,5.0,6882.0,5.0,5.0,0.0,8


In [45]:
product_list = ['아디다스 삼바 OG 클라우드 화이트', '반스 올드스쿨 블랙', 
                '뉴발란스 993 메이드 인 USA 그레이', '컨버스 척 70 하이 블랙', '컨버스 척 70 로우 블랙', 
                '조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루', "나이키 에어포스 1 '07 WB 플랙스", 
                '팀버랜드 프리미엄 6인치 워터프루프 부츠 위트 누벅', '미하라 야스히로 피터슨 OG 솔 캔버스 로우 스니커즈 블랙', 
                '아디다스 슈퍼스타 코어 블랙 화이트', "나이키 에어포스 1 '07 로우 트리플 블랙", '뉴발란스 992 메이드 인 USA 그레이',
                  "나이키 에어포스 1 '07 로우 화이트", '나이키 덩크 로우 레트로 블랙', '아디다스 삼바 OG 블랙 화이트 검']

In [46]:
# 여성 제품과 남성 제품을 통일
data.loc[data['name_kor'].str.startswith('(W)', na=False), 'name_kor'] = data['name_kor'].str.replace(r'^\(W\)\s*', '', regex=True)

product_data = data[data['name_kor'].apply(lambda x: isinstance(x, str) and any(product in x for product in product_list))]

# name_kor 별로 그룹화하여 create_to_purchase_duration의 평균 계산 및 행의 개수 계산

grouped_data = product_data.groupby('name_kor').agg(
    average_duration=('create_to_purchase_duration', 'mean'),
    count=('create_to_purchase_duration', 'size')
).reset_index()

# 평균값 기준으로 내림차순 정렬
sorted_grouped_data = grouped_data.sort_values(by='average_duration', ascending=False)

# 결과 출력
sorted_grouped_data

,name_kor,average_duration,count
11,반스 올드스쿨 블랙 검,31327.375000,8
10,반스 올드스쿨 블랙,23703.228070,57
16,컨버스 척 70 로우 블랙,17564.661290,62
17,컨버스 척 70 하이 블랙,15153.789474,76
13,아디다스 삼바 OG 클라우드 화이트,14868.775000,80
0,나이키 덩크 로우 레트로 블랙,13637.955224,67
14,아디다스 슈퍼스타 코어 블랙 화이트,12847.682243,107
12,아디다스 삼바 OG 블랙 화이트 검,12357.460317,63
4,나이키 에어포스 1 '07 로우 화이트,11015.765000,200
9,미하라 야스히로 피터슨 OG 솔 캔버스 로우 스니커즈 블랙,10098.116279,43


In [47]:
# 기준 값 설정
reference_value = sorted_grouped_data[sorted_grouped_data["name_kor"] == "나이키 에어포스 1 '07 로우 화이트"]["average_duration"].iloc[0]

# ±30% 범위 계산
lower_bound = reference_value * 0.7
upper_bound = reference_value * 1.3

# 범위 내의 값 필터링
filtered_df = sorted_grouped_data[(sorted_grouped_data["average_duration"] >= lower_bound) & (sorted_grouped_data["average_duration"] <= upper_bound)]


In [48]:
filtered_df['name_kor'].unique()

array(['나이키 덩크 로우 레트로 블랙', '아디다스 슈퍼스타 코어 블랙 화이트', '아디다스 삼바 OG 블랙 화이트 검',
       "나이키 에어포스 1 '07 로우 화이트", '미하라 야스히로 피터슨 OG 솔 캔버스 로우 스니커즈 블랙',
       "나이키 에어포스 1 '07 로우 트리플 블랙", '뉴발란스 993 메이드 인 USA 그레이',
       "나이키 에어포스 1 '07 WB 플랙스"], dtype=object)

## 모델 선정

In [49]:
selected_model=['나이키 덩크 로우 레트로 블랙', '아디다스 슈퍼스타 코어 블랙 화이트', '아디다스 삼바 OG 블랙 화이트 검',
       "나이키 에어포스 1 '07 로우 화이트", '미하라 야스히로 피터슨 OG 솔 캔버스 로우 스니커즈 블랙',
       "나이키 에어포스 1 '07 로우 트리플 블랙", '뉴발란스 993 메이드 인 USA 그레이',
       "나이키 에어포스 1 '07 WB 플랙스"]

In [50]:
df_no_duplicates = data.drop_duplicates(subset=['id'], keep=False)

In [51]:
df_filtered_products = df_no_duplicates[df_no_duplicates['name_kor'].isin(selected_model)]
df_filtered_products

,id,status,create_date,name,keyword,price,flag_used,flag_taekpo,flag_exchg,qty,...,pid,first_view_date,purchase_date,create_to_view_duration,create_to_talk_duration,create_to_purchase_duration,view_to_talk_duration,view_to_purchase_duration,talk_to_purchase_duration,total_view
73,3ef9b871794aad8e8b9a7ecff4198005,3,2023-12-11 19:45:00,에어포스 블랙 275,"에어포스275,나이키,나이키에어포스블랙",80000,21,0,0,0,...,3ef9b871794aad8e8b9a7ecff4198005,2023-12-11,2023-12-15 01:25:00,0.0,1.0,4660.0,1.0,4.0,3.0,137
76,9d102d7cb403e706afd72802c569c4f6,3,2024-03-05 20:59:00,[260] 뉴발란스 993 그레이,"뉴발란스,993,260",300000,20,1,0,0,...,9d102d7cb403e706afd72802c569c4f6,2024-03-05,2024-03-13 15:51:00,0.0,1.0,11212.0,1.0,8.0,7.0,67
79,9529223602a6babb9620fdbf3d2e9d12,3,2023-12-31 07:11:00,[265] 나이키 범고래 덩크로우 레트로,"나이키,덩크로우,범고래,덩크,조던",35000,22,1,0,0,...,9529223602a6babb9620fdbf3d2e9d12,2023-12-31,2024-01-04 15:29:00,0.0,0.0,6258.0,0.0,4.0,4.0,64
95,fa317e264af77cf10675c52f84c4dbfc,3,2024-01-17 11:27:00,뉴발993 gray 2E 255size,"뉴발993,뉴발란스993,뉴발993그레이",310000,2,0,0,0,...,fa317e264af77cf10675c52f84c4dbfc,2024-01-17,2024-01-23 02:38:00,0.0,2.0,8111.0,2.0,6.0,4.0,5
140,4720e3f8f4b48ebf54867980f3e3431c,3,2024-04-28 20:27:00,뉴발란스 993 8d(260),NaN,160000,21,1,0,0,...,4720e3f8f4b48ebf54867980f3e3431c,2024-04-28,2024-05-03 06:28:00,0.0,5.0,6361.0,5.0,5.0,0.0,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20339,db0564ef098358e074b1f8c3b212f7ad,2,2024-03-02 17:10:00,나이키 덩크로우 범고래 255 사이즈 판매합니다,"나이키,덩크로우,범고래,나이키범고래,덩크로우범고래",72000,20,1,0,0,...,db0564ef098358e074b1f8c3b212f7ad,2024-03-02,2024-03-04 13:10:00,0.0,2.0,2640.0,2.0,2.0,0.0,20
20343,5bc19bead06de966c989b4a198535030,3,2023-12-24 23:03:00,[245 새상품] 아디다스 삼바 OG,"삼바,아디다스,삼바245",130000,2,0,0,0,...,5bc19bead06de966c989b4a198535030,2023-12-25,2024-01-06 07:18:00,1.0,12.0,17775.0,11.0,12.0,1.0,32
20373,c55fb5c7e1120bb00043fcb61e9bb2ac,3,2023-12-08 19:26:00,아디다스 슈퍼스타 270,"아디다스슈퍼스타,슈퍼스타",55000,22,1,0,0,...,c55fb5c7e1120bb00043fcb61e9bb2ac,2023-12-08,2023-12-09 14:55:00,0.0,1.0,1169.0,1.0,1.0,0.0,16
20415,05c53158a3cdbed45acc738d49ad113f,3,2024-02-13 14:16:00,새상품)나이키 에어포스 275,"나이키,에어포스,에어포스275,275",125000,2,0,0,0,...,05c53158a3cdbed45acc738d49ad113f,2024-02-13,2024-02-14 10:08:00,0.0,0.0,1192.0,0.0,1.0,1.0,12


In [52]:
#판매 기간을 위한 데이터 변환
df_filtered_products['setl_done_date'] = pd.to_datetime(df_filtered_products['setl_done_date'], errors='coerce')
df_filtered_products['purchase_confirm_date'] = pd.to_datetime(df_filtered_products['purchase_confirm_date'], errors='coerce')
df_filtered_products['final_purchase_date'] = df_filtered_products[['setl_done_date', 'purchase_confirm_date']].min(axis=1)
#판매 기간 계산
df_filtered_products['create_date'] = pd.to_datetime(df_filtered_products['create_date'], errors='coerce')
df_filtered_products['first_talk_date'] = pd.to_datetime(df_filtered_products['first_talk_date'], errors='coerce')
df_filtered_products['purchase_duration'] = (df_filtered_products['final_purchase_date'] - df_filtered_products['create_date']).dt.total_seconds() / 3600
#df_cleaned['purchase_duration'] = (df_cleaned['final_purchase_date'] - df_cleaned['create_date']).dt.days
df_filtered_products['talk_duration'] = (df_filtered_products['first_talk_date'] - df_filtered_products['create_date']).dt.days


C:\Users\sunwoong\AppData\Local\Temp\ipykernel_30168\2513117353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_products['setl_done_date'] = pd.to_datetime(df_filtered_products['setl_done_date'], errors='coerce')
C:\Users\sunwoong\AppData\Local\Temp\ipykernel_30168\2513117353.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_products['purchase_confirm_date'] = pd.to_datetime(df_filtered_products['purchase_confirm_date'], errors='coerce')
C:\Users\sunwoong\AppData\Local\Temp\ipy

In [54]:
df_filtered_products.to_csv('0819_keyword_analysis.csv',encoding='utf-8',index=False)

In [55]:
df_filtered_products.to_csv('0819_keyword_analysis_cp949.csv',encoding='cp949',index=False)